In [12]:
from recommendations import critics
import recommendations

In [2]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [3]:
critics['Toby']['Snakes on a Plane']=4.5

In [4]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

In [5]:
#for example calculate the distance between two point
from math import sqrt
sqrt(pow(5-4,2)+pow(4-1,2))

3.1622776601683795

In [6]:
1/(1+sqrt(pow(5-4,2)+pow(4-1,2)))

0.2402530733520421

# Euclidean Distance Score

In [7]:
##Make a function to wrap them up
#The code for sim_distance
from math import sqrt
#Return a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    #Get the list of shared_items
    si = {}
    for item in prefs[preson1]:
        if item in prefs[person2]:
            si[item] = 1
#if they have no ratings in common, return 0
    if len(si)==0: return 0
#Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2)
                          for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

In [17]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [18]:
recommendations.sim_distance(recommendations.critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

# Pearson Correlation Score

In [19]:
#Return the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    #Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:si[item]=1
    #Find the number of elements
    n = len(si)
    #If they are no ratings in common, return 0
    if n==0:return 0
    #Add yup all tje preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    #Sum up the squares 
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    #sum up the products
    pSum = sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    #Calculate Pearson score
    num = pSum-(sum1*sum2/n)
    den = sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0:return 0
    r = num/den
    return r

In [31]:
reload(recommendations)
recommendations.sim_pearson(recommendations.critics,
                           'Lisa Rose','Gene Seymour')

0.39605901719066977

In [32]:
###Ranking the Critics

In [ ]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.

In [33]:
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)
           for other in prefs if other!=person]
    #sort the list so the highest scores appear at the top
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [38]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [39]:
recommendations.topMatches(recommendations.critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

In [ ]:
##Recommending Items

In [40]:
#Gets recommendations for a person by using a weighted average
#of every other user;s rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        #don't compare me to myself
        if other == person:continue
        sim = similarity(prefs,person,other)
        #ignore scores of zero or lower
        if sim<=0:continue
        for item in prefs[other]:
            #only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                #Similarity*score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                #Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
        #Creating the normalized list
        rankings=[(total/simSums[item],item) for item,total in totals.items()]
        #Return the sorted list
        rankings.sort()
        rankings.reverse()
        return rankings

In [47]:
reload(recommendations)
pass

In [48]:
recommendations.getRecommendations(recommendations.critics,'Toby')

[(3.0, 'The Night Listener'), (3.0, 'Lady in the Water')]

In [49]:
recommendations.getRecommendations(recommendations.critics,'Toby',
                similarity=recommendations.sim_distance)

[(2.9999999999999996, 'The Night Listener'),
 (2.9999999999999996, 'Lady in the Water')]

In [64]:
###Matching Products
#Swap the people to items

In [65]:
'''
{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}
to:
 {'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}
'''

"\n{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},\n    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}\nto:\n {'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},\n'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}\n"

In [ ]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
        #Flip them and person
        result[item][person]=prefs[person][item]
    return result

In [72]:
reload(recommendations)
pass

In [73]:
movies = recommendations.transformPrefs(recommendations.critics)
recommendations.topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [74]:
#To twist things around even more, you can get recommended critics for a movie.

In [75]:
recommendations.getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [76]:
##The del.icio.us API

In [1]:
import pydelicious

In [2]:
pydelicious.get_popular(tag='python')

[{'count': '',
  'description': u'something went wrong',
  'dt': '',
  'extended': '',
  'hash': '',
  'href': '',
  'tags': '',
  'user': ''}]

In [1]:
#Item-Based Filtering
#In cases with very large datasets, item-based collaborative filtering can give better results, 
#and it allows many of the calculations to be 
#performed in advance so that a user needing recommenda- tions can get them more quickly.

In [2]:
def calculateSimilarItems(prefs,n=10):
    #Create a dictionary of items showing which other items they
    #are most similar to 
    result={}
    #Invert the preference matrix to be item-centric
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        #Status updates for large datasets
        c+=1
        if c%100==0:print "%d/%d"%(c,len(itemPrefs))
        #Find the most similar items on this one
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [4]:
reload(recommendations)

In [6]:
itemsim=recommendations.calculateSimilarItems(recommendations.critics)

In [7]:
itemsim

{'Just My Luck': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'You, Me and Dupree'),
  (0.15384615384615385, 'The Night Listener'),
  (0.10526315789473684, 'Snakes on a Plane'),
  (0.06451612903225806, 'Superman Returns')],
 'Lady in the Water': [(0.4, 'You, Me and Dupree'),
  (0.2857142857142857, 'The Night Listener'),
  (0.2222222222222222, 'Snakes on a Plane'),
  (0.2222222222222222, 'Just My Luck'),
  (0.09090909090909091, 'Superman Returns')],
 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'The Night Listener'),
  (0.16666666666666666, 'Superman Returns'),
  (0.10526315789473684, 'Just My Luck'),
  (0.05128205128205128, 'You, Me and Dupree')],
 'Superman Returns': [(0.16666666666666666, 'Snakes on a Plane'),
  (0.10256410256410256, 'The Night Listener'),
  (0.09090909090909091, 'Lady in the Water'),
  (0.06451612903225806, 'Just My Luck'),
  (0.05333333333333334, 'You, Me and Dupree')],
 'The Night Listener': [(0.28

In [8]:
##Getting Recommendations

In [ ]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    #Loop over items rated by this user
    for (item,rating)in userRatings.items():
        #Loop over item similar to this one
        for(similarity,items)in itemMatch[item]:
            #Ignore if this user has already rated this item
            if item2 in userRatings:continue
            #Weighted sum of rating tiems similarity 
            scores.setdefault(items,0)
            scores[items]+=similarity*rating
            #Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
    #Divide each total score by total weighting to get an average
    rankings = [(score/totalSim[item],item) for item,score in scores.items()]
    #Return the rankings from highest to lowest
    rankings.sort()
    rankings.reverse()
    return rankings

In [13]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [14]:
recommendations.getRecommendedItems(recommendations.critics,itemsim,'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

In [15]:
#Using the MovieLens Dataset

In [ ]:
def loadMovieLens(path='/ml-100k'):
    #Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    #Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [38]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [34]:
prefs=recommendations.loadMovieLens()

In [35]:
prefs['87']

{'2001: A Space Odyssey (1968)': 5.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Addams Family Values (1993)': 2.0,
 'Addicted to Love (1997)': 4.0,
 'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Air Force One (1997)': 3.0,
 'Air Up There, The (1994)': 3.0,
 'Alien (1979)': 4.0,
 'American President, The (1995)': 5.0,
 'Annie Hall (1977)': 4.0,
 'Apocalypse Now (1979)': 4.0,
 'Babe (1995)': 5.0,
 'Baby-Sitters Club, The (1995)': 2.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 4.0,
 'Bananas (1971)': 5.0,
 'Barcelona (1994)': 3.0,
 'Batman & Robin (1997)': 4.0,
 'Batman (1989)': 3.0,
 'Batman Returns (1992)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Big Squeeze, The (1996)': 2.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 4.0,
 'Blues Brothers, The (1980)': 5.0,
 'Boomerang (1992)': 3.0,
 'Boot, Das (1981)': 4.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Braveheart (1995)': 4.0,
 'Bridge on the River

In [36]:
recommendations.getRecommendations(prefs,'87')[0:30]

[(5.0, 'Wrong Trousers, The (1993)'),
 (5.0, 'Willy Wonka and the Chocolate Factory (1971)'),
 (5.0, 'When We Were Kings (1996)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Shall We Dance? (1996)'),
 (5.0, 'Secrets & Lies (1996)'),
 (5.0, 'Roman Holiday (1953)'),
 (5.0, 'Princess Bride, The (1987)'),
 (5.0, 'Postino, Il (1994)'),
 (5.0, 'Persuasion (1995)'),
 (5.0, "One Flew Over the Cuckoo's Nest (1975)"),
 (5.0, 'Nikita (La Femme Nikita) (1990)'),
 (5.0, 'My Life as a Dog (Mitt liv som hund) (1985)'),
 (5.0, 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)'),
 (5.0, 'Maya Lin: A Strong Clear Vision (1994)'),
 (5.0, 'Lone Star (1996)'),
 (5.0, 'Local Hero (1983)'),
 (5.0, 'L.A. Confidential (1997)'),
 (5.0, 'Henry V (1989)'),
 (5.0, 'Good Will Hunting (1997)'),
 (5.0, 'Gaslight (1944)'),
 (5.0, 'Eat Drink Man Woman (1994)'),
 (5.0, 'Close Shave, A (1995)'),
 (5.0, 'Brazil (1985)'),
 (5.0, 'Big Night (1996)'),
 (5.0, 'Being There (1979)'),
 (5.0, 'Beauty and the Beast (1991)'),
 (5.0, 

In [39]:
itemsim = recommendations.calculateSimilarItems(prefs,n=50)

100/1664
200/1664
300/1664
400/1664
500/1664
600/1664
700/1664
800/1664
900/1664
1000/1664
1100/1664
1200/1664
1300/1664
1400/1664
1500/1664
1600/1664


In [40]:
recommendations.getRecommendations(prefs,'87')[0:30]

[(5.0, 'Wrong Trousers, The (1993)'),
 (5.0, 'Willy Wonka and the Chocolate Factory (1971)'),
 (5.0, 'When We Were Kings (1996)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Shall We Dance? (1996)'),
 (5.0, 'Secrets & Lies (1996)'),
 (5.0, 'Roman Holiday (1953)'),
 (5.0, 'Princess Bride, The (1987)'),
 (5.0, 'Postino, Il (1994)'),
 (5.0, 'Persuasion (1995)'),
 (5.0, "One Flew Over the Cuckoo's Nest (1975)"),
 (5.0, 'Nikita (La Femme Nikita) (1990)'),
 (5.0, 'My Life as a Dog (Mitt liv som hund) (1985)'),
 (5.0, 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)'),
 (5.0, 'Maya Lin: A Strong Clear Vision (1994)'),
 (5.0, 'Lone Star (1996)'),
 (5.0, 'Local Hero (1983)'),
 (5.0, 'L.A. Confidential (1997)'),
 (5.0, 'Henry V (1989)'),
 (5.0, 'Good Will Hunting (1997)'),
 (5.0, 'Gaslight (1944)'),
 (5.0, 'Eat Drink Man Woman (1994)'),
 (5.0, 'Close Shave, A (1995)'),
 (5.0, 'Brazil (1985)'),
 (5.0, 'Big Night (1996)'),
 (5.0, 'Being There (1979)'),
 (5.0, 'Beauty and the Beast (1991)'),
 (5.0, 